https://github.com/IgnacioPardo/Tecnologias_Exponenciales_2023/blob/main/Consigna_NN.md
https://github.com/IgnacioPardo/SNN/tree/main

# Trabajo Práctico Redes Neuronales

## Introducción

El objetivo de este trabajo es plantear una red neuronal con una capa de entrada con $n$ neuronas y una capa de salida con $m$ salidas para resolver un problema de su propía creación.

Se evaluara la correcta implementación de la red neuronal y la explicación de los elementos de la teoría vistos en clase.

## Consigna

A partir del siguiente Notebook visto en clase [SNN](https://github.com/IgnacioPardo/SNN/blob/main/SNN.ipynb), se pide:

1. Plantear un problema a resolver con una red neuronal de una capa de entrada y una capa de salida. Las salidas no deben ser excluyentes, es decir, pueden pertenecer a mas de una clase. Algunos ejemplos son: A partir de caracteristicas de una vivienda, determinar si la vivienda es de algun estilo (moderna, agradable, tradicional). Pueden generar ustedes un pequeño dataset para utilizar o buscar en Kaggle.
2. Modificar la estructura de la red neuronal para que admita $n$ neuronas de entrada y $m$ neuronas de salida.

Ademas deben indicar los elementos de la teoría que vimos en clase que identifiquen en el código y explicarlos (Estructura de la Red, Función de activación, Forward propagation, Back propagation y Gradiente descendente).

## Bonus

A. ¿Como modificarían la estructura de la red para que admita $k$ capas ocultas con $n$ neuronas cada una? Describir como sería el algoritmo de forward propagation en este caso, no es necesario implementar el algoritmo de back propagation.

B. Investigar la función de activación Softmax y explicar como se puede utilizar para resolver un problema de clasificación con $m$ clases mutualmente excluyentes. ¿Que habría que modificar en el código para utilizar esta función de activación?

## Entrega

La entrega consiste en un notebook de Jupyter con el código y las explicaciones correspondientes. La fecha de entrega es el 29/10/2023 a través del siguiente [Google Forms]().

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML, display
import tabulate
import networkx as nx
from random import random as rand
import itertools

In [ ]:
def sigmoid(x):
    return 1 / ( 1 + np.exp(-x) )

# sigmoid = lambda x : 1 / ( 1 + np.exp(-x) )

def sigmoid_derivative(x):
    return x * (1 - x)

In [ ]:
df = pd.read_csv("seattle-weather.csv")
df = df.drop(columns=['date'])
weather_tof = pd.get_dummies(df['weather'])
df = pd.concat([df, weather_tof], axis=1)
df